# Contact Center Analytics and Improvement using Azure OpenAI Service

## Extract rich insights from call transcripts


Contact center analytics and improvement using Azure OpenAI services allows businesses to gain deep insights from the recordings of conversations that have taken place between customers and contact center agents. These recordings are processed using natural language processing techniques, and the data extracted is then analyzed to gain insight into the call topics, customer sentiment, and the effectiveness of contact center agents. This information can then be used to improve customer experiences, streamline processes, and identify areas of potential improvement. 
Additionally, Azure OpenAI services can be used to train automated systems with the insights gained from the recorded customer conversations, which helps in providing real-time assistance to customers quickly and efficiently.

### Setup global variables 

In [10]:
import openai
import os
import numpy as np
import pandas as pd
from openai import cli
import json

# Load config values
with open(r'config.json') as config_file:
    config_details = json.load(config_file)

# Setting up the deployment name
deployment_name = config_details['COMPLETIONS_MODEL']

# This is set to `azure`
openai.api_type = "azure"

# The API key for your Azure OpenAI resource.
API_KEY = config_details['OPENAI_API_KEY']
openai.api_key = os.getenv(API_KEY)
openai.api_key = API_KEY

# The base URL for your Azure OpenAI resource. e.g. "https://<your resource name>.openai.azure.com"
openai.api_base = config_details['OPENAI_API_BASE']

# Currently OPENAI API have the following versions available: 2022-12-01
openai.api_version = config_details['OPENAI_API_VERSION']

ENGINE = config_details['OPENAI_ENGINE_DAVINCI']

# Analytics

### Customer and agent conversation 

In [7]:
transcript = """
Agent: Hi there! Thank you for calling Wide World Importers. How may I help you today?
Customer: Could you suggest some solar panels for my new home.
Agent: Sure! We offer free solar panel analysis for your home to determine what kind of solar panel you need? Would you like to request a free solar panel analysis?
Customer: Yes, I'd like a free solar panel analysis.
Agent: Awesome! Let's get started. Can you give me your street address?
Customer: SW 61st Ave.
Agent: What's your ZIP code?
Customer: FL 33314
Agent: Okay, let me pull up your information… Based on the average energy consumption in your neighbourhood, we recommend Relecloud 150 (RC-MX100-800). It comes with excellent panels compatible with the most portable power stations. 
It is an 800-Watt 12-Volt Off-Grid Solar Premium Kit w/ 8-Piece 100W Monocrystalline Panel and 60A MPPT Rover Charge Controller. 
You can also check our bestselling solar panels Relecloud 100 and Relecloud 125. They have been rated 4.5/5 by our customers.
Customer: Thanks. That's very helpful.
Agent: No problem. Is there anything else I may assist you with?
Customer: No, thank you. I will look at the options suggested by you.
Agent:  Sure. Wish you a great day!
"""

### Instructions for the Azure OpenAI Service 

In [8]:
instructions = """Extract the following from the conversation:
1. What was the main reason of the conversation?
2. what was the overall sentiment of the customer?
3. How did the agent handle the conversation?
4. what was the final outcome of the conversation?
5. Create a short summary of the conversation
6. any Action items for follow-up?
"""

In [11]:
def transcript_analysis(transcript, instructions):
    prompt = transcript + instructions
    response = openai.Completion.create(
      engine=ENGINE,
        prompt= prompt,
        temperature=1,
        max_tokens=2500,
        top_p=0.5,
        frequency_penalty=0,
        presence_penalty=0,
        best_of=1,
        stop=None
  )
    res = response['choices'][0]['text']
    print(res)
    
transcript_analysis(transcript, instructions)


1. The main reason of the conversation was to suggest solar panels for the customer's new home.
2. The customer was satisfied with the help they received.
3. The agent handled the conversation professionally and provided helpful information.
4. The final outcome of the conversation was that the customer thanked the agent and said they would look at the options suggested.
5. The customer called Wide World Importers for help in finding solar panels for their new home. The agent provided helpful information and suggested Relecloud 150, Relecloud 100, and Relecloud 125 as options. The customer thanked the agent and said they would look at the options suggested.
6. No action items for follow-up.


# Improvement

In [12]:
prompt = """
Employee: Hi there! Welcome to Wide World Importers. How may I help you today?

Customer: Could you suggest some solar panels for my new home.

Employee: Sure! We offer free solar panel analysis for your home to determine what kind of solar panel you need? Would you like to request a free solar panel analysis?

Customer: Yes, I'd like a free solar panel analysis.

Employee: Awesome! Let's get started. What's your street address?

Customer: SW 61st Ave.

Employee: What's your ZIP code?

Customer: FL 33314

Employee: Okay, let me pull up your information…
Based on the average energy consumption in your neighbourhood, we recommend Relecloud 150 (RC-MX100-800). It comes with excellent panels compatible with the most portable power stations. It is an 800-Watt 12-Volt Off-Grid Solar Premium Kit w/ 8-Piece 100W Monocrystalline Panel and 60A MPPT Rover Charge Controller. You can also check our bestselling solar panels Relecloud 100 and Relecloud 125. They have been rated 4.5/5 by our customers.
 
Customer: Thanks. That’s very helpful.

Employee: No problem. Is there anything else I may assist you with?

Customer: No, thank you. I will look at the options suggested by you.

Employee:  Sure. Wish you a great day!
"""

In [13]:
instruction ="improve the above transcript:"

In [14]:
def transcript_betterment(prompt):
    prompt = prompt + instruction
    response = openai.Completion.create(
      engine=ENGINE,
        prompt= prompt,
        temperature=0.9,
        max_tokens=2500,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        best_of=1,
        stop=None
  )
    res = response['choices'][0]['text']
    print(res)
    
transcript_betterment(prompt)



Employee: Hi there! Welcome to Wide World Importers. How can I help you today?

Customer: Could you suggest some solar panels for my new home?

Employee: Absolutely! We offer free solar panel analysis for your home to determine what kind of solar panel you need. Would you like to request a free solar panel analysis?

Customer: Yes, please.

Employee: Great! Let me get some information from you. What's your street address?

Customer: SW 61st Ave.

Employee: And what's your ZIP code?

Customer: FL 33314

Employee: Got it. Let me pull up your information… Based on the average energy consumption in your neighbourhood, we recommend Relecloud 150 (RC-MX100-800). It comes with excellent panels compatible with the most portable power stations. It is an 800-Watt 12-Volt Off-Grid Solar Premium Kit w/ 8-Piece 100W Monocrystalline Panel and 60A MPPT Rover Charge Controller. You can also check our bestselling solar panels Relecloud 100 and Relecloud 125. They have been rated 4.5/5 by our customer

# Summary and Issue Resolution

## Natural Language

In [15]:
def get_resolution(text):
    add_text = """Generate a summary of the below given conversation in the following format:
                  Customer problem:
                  Outcome of the conversation:
                  Action items for follow-up:
                  Customer budget:
                  Departure city:
                  Destination city:
                  
                  Conversation:\n"""
    prompt = add_text + '\n' + text + '\n' + 'Summary:' + '\n'
    response = openai.Completion.create(
      engine=ENGINE,
        prompt= prompt,
        temperature=0.7,
        max_tokens=1500,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        best_of=1,
        stop=None
  )
    res = response['choices'][0]['text']
    print(res)

In [16]:
text = """User: Hi there, I’m off between August 25 and September 11. I saved up 4000 for a nice trip. If I flew out from San Francisco, what are your suggestions for where I can go?
          Agent: For that budget you could travel to cities in the US, Mexico, Brazil, Italy or Japan. Any preferences?
          User: Excellent, I’ve always wanted to see Japan. What kind of hotel can I expect?
          Agent: Great, let me check what I have. First, can I just confirm with you that this is a trip for one adult?
          User: Yes it is
          Agent: Great, thank you, In that case I can offer you 15 days at HOTEL Sugoi, a 3 star hotel close to a Palace. You would be staying there between August 25th and September 7th. They offer free wifi and have an excellent guest rating of 8.49/10. The entire package costs 2024.25USD. Should I book this for you?
          User: That sounds really good actually. Please book me at Sugoi.
          Agent: I can do that for you! Can I help you with anything else today?
          User: No, thanks! Please just send me the itinerary to my email soon."""

get_resolution(text)

Customer problem: The customer was looking for a vacation destination and hotel within their budget of $4000.
Outcome of the conversation: The customer chose to stay at the Hotel Sugoi in Japan for 15 days, costing $2024.25. 
Action items for follow-up: The agent will send the customer an itinerary to their email. 
Customer budget: $4000 
Departure city: San Francisco 
Destination city: Japan


## Machine Readable

In [17]:
instruction = """Extract the following information from the conversation below:   

Call reason (key: reason)  
Caller name (key: caller_name)  
Agent Name (key: agent_name)  
Truck location (key: location)  
Vin (key: VIN)  
Caller sentiment (key: caller_sentiment)  
Fault code (key: fault_code)  
A short, yet detailed summary (key: summary)  

Please answer in JSON machine-readable format, using the keys from above. Format the output as a JSON object called “results”. Pretty print the JSON and make sure that it is properly closed at the end.  """

prompt = instruction + transcript

In [19]:
def get_resolution_machine_readable(prompt):
    prompt = prompt
    response = openai.Completion.create(
      engine=ENGINE,
        prompt= prompt,
        temperature=0.7,
        max_tokens=1500,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        best_of=1,
        stop=None
  )
    res = response['choices'][0]['text']
    print(res)

get_resolution_machine_readable(prompt)


results = {
  "reason": "Request a free solar panel analysis",
  "caller_name": "Customer",
  "agent_name": "Agent",
  "location": "SW 61st Ave., FL 33314",
  "VIN": null,
  "caller_sentiment": "Thanks. That's very helpful.",
  "fault_code": null,
  "summary": "The customer requested a free solar panel analysis and the Agent provided information about Relecloud 150, Relecloud 100 and Relecloud 125 as the bestsellers. The customer thanked the Agent for the help."
}
